In [ ]:
import glob

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np 
import pandas as pd 

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import InceptionV3, DenseNet201, ResNet50, EfficientNetB6
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras.layers import Flatten, Concatenate
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
import os


In [ ]:
batch_size = 256
IMAGE_SIZE = [512, 512]
num_epochs = 80

In [ ]:
# Set distribution strategy to use TPUs
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
print('Found connected TPU: ', resolver.cluster_spec().as_dict()['worker'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
from kaggle_datasets import KaggleDatasets

GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(GCS_DS_PATH) 

In [ ]:
train_files = tf.io.gfile.glob(f'{GCS_DS_PATH}/tfrecords-jpeg-512x512/train/*')
val_files = tf.io.gfile.glob(f'{GCS_DS_PATH}/tfrecords-jpeg-512x512/val/*')
test_files = tf.io.gfile.glob(f'{GCS_DS_PATH}/tfrecords-jpeg-512x512/test/*')

In [ ]:
def get_num_samples(file_list):
    count = 0 
    for file_name in file_list:
        num_sample = int(file_name.split('.tfrec')[0].rsplit('-', 1)[1])
        count += num_sample
    return count

In [ ]:
train_size = get_num_samples(train_files)
val_size = get_num_samples(val_files)
test_size = get_num_samples(test_files)

print(f"Train dataset size: {train_size}")
print(f"Validation dataset size: {val_size}")
print(f"Test dataset size: {test_size}")

In [ ]:
# Functions and 'classes' variable in this cell were taken from https://www.kaggle.com/code/ryanholbrook/create-your-first-submission/notebook

CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102

def data_augment(image, label):
    # Thanks to the dataset.prefetch(AUTO)
    # statement in the next function (below), this happens essentially
    # for free on TPU. Data pipeline code is executed on the "CPU"
    # part of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label   

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

In [ ]:
train_dataset = tf.data.TFRecordDataset(train_files, num_parallel_reads=AUTO)
test_dataset = tf.data.TFRecordDataset(test_files, num_parallel_reads=AUTO)
val_dataset = tf.data.TFRecordDataset(val_files, num_parallel_reads=AUTO)

train_dataset = train_dataset.map(read_labeled_tfrecord)
train_dataset = train_dataset.map(data_augment, num_parallel_calls=AUTO)
train_dataset = train_dataset.shuffle(buffer_size=batch_size)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(batch_size=batch_size)
train_dataset = train_dataset.prefetch(buffer_size=batch_size)

val_dataset = val_dataset.map(read_labeled_tfrecord)
val_dataset = val_dataset.batch(batch_size=batch_size)
val_dataset = val_dataset.prefetch(buffer_size=batch_size)

test_dataset = test_dataset.map(read_unlabeled_tfrecord)
test_dataset = test_dataset.prefetch(buffer_size=batch_size)

In [ ]:
def create_densenet201_model():
    base_model = DenseNet201(weights="imagenet", include_top=False, input_tensor=Input(shape=(512, 512, 3)))
    head_model = base_model.output
    head_model = GlobalAveragePooling2D()(head_model)
    head_model = Flatten(name="flatten")(head_model)
#     head_model = Dense(4096, activation="relu")(head_model)
#     head_model = Dropout(0.4)(head_model)
    head_model = Dense(len(CLASSES), activation="softmax")(head_model)

    # Place the head FC model on top of the base model (this will become the actual model we will train)
    model = Model(inputs=base_model.input, outputs=head_model)
    for layer in base_model.layers:
        layer.trainable = True
#     opt = Adam(learning_rate=0.0001)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['sparse_categorical_accuracy'])
    model.summary()
    return model

In [ ]:
# def desenet121_model():
#     base_model = EfficientNetB6(weights="imagenet", include_top=False, input_tensor=Input(shape=(512, 512, 3)))
#     head_model = base_model.output
#     head_model = GlobalAveragePooling2D()(head_model)
#     head_model = Flatten(name="flatten")(head_model)
# #     head_model = Dense(2048, activation="relu")(head_model)
# #     head_model = Dropout(0.5)(head_model)
#     head_model = Dense(len(CLASSES), activation="softmax")(head_model)

#     # Place the head FC model on top of the base model (this will become the actual model we will train)
#     model = Model(inputs=base_model.input, outputs=head_model)
#     for layer in base_model.layers:
#         layer.trainable = True
# #     opt = Adam(learning_rate=0.0001)
#     model.compile(optimizer='adam',
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['sparse_categorical_accuracy'])
#     model.summary()
#     return model

In [ ]:
# def nerual_network_model():
#     base_model = DenseNet201(weights="imagenet", include_top=False, input_tensor=Input(shape=(512, 512, 3)))
#     head_model = base_model.output
#     head_model = GlobalAveragePooling2D()(head_model)
#     head_model = Flatten(name="flatten1")(head_model)
#     head_model = Dense(4096, activation="relu")(head_model)
#     head_model = Dropout(0.5)(head_model)
#     head_model = Dense(len(CLASSES), activation="softmax")(head_model)
#     model_1 = Model(inputs=base_model.input, outputs=head_model)


#     base_model = InceptionV3(weights="imagenet", include_top=False, input_tensor=Input(shape=(512, 512, 3)))
#     head_model = base_model.output
#     head_model = GlobalAveragePooling2D()(head_model)
#     head_model = Flatten(name="flatten2")(head_model)
#     head_model = Dense(4096, activation="relu")(head_model)
#     head_model = Dropout(0.5)(head_model)
#     head_model = Dense(len(CLASSES), activation="softmax")(head_model)
#     model_2 = Model(inputs=base_model.input, outputs=head_model)
    
#     x = Concatenate()([model_1.output, model_2.output])
#     x = Dense(len(CLASSES), activation='softmax')(x)
#     model = Model(inputs=[model_1.input,model_2.input],outputs=x)
# #     model.summary()
#     opt = Adam(learning_rate=0.0001)
#     model.compile(optimizer=opt,
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['sparse_categorical_accuracy'])
#     return model

In [ ]:
# def nerual_network_model():
#     inception_model =DenseNet121(weights="imagenet", include_top=False, input_shape=(512,512,3))
#     x = inception_model.output
#     x = GlobalAveragePooling2D()(x)
#     x = Flatten()(x)
# #     x = Dense(4096, activation='relu')(x)
# #     x = Dropout(0.5)(x)
# #     predictions=Dense(len(CLASSES), activation='softmax')(x)
    
#     #model_1 = Model(inputs=vgg16_model.input, outputs=predictions)
#     model_1 = Model(inputs=inception_model.input, outputs=x)
#     for layer in inception_model.layers:
#         layer.trainable = False
        
#     densenet201_model = DenseNet201(weights="imagenet", include_top=False, input_shape=(512,512,3))#Edited here.Using same model causes name conflicts of layers.Every layer names should be unique.
#     x = densenet201_model.output
#     #resnet50_model =ResNet50(weights="imagenet", include_top=False, input_shape=(224,224,3))
#     #x = resnet50_model.output
#     x = GlobalAveragePooling2D()(x)
#     x = Flatten()(x)
# #     x = Dense(4096, activation='relu')(x)
# #     x = Dropout(0.5)(x)
# #     predictions=Dense(len(CLASSES), activation='softmax')(x)

#     model_2 = Model(inputs=densenet201_model.input, outputs=x)
#     for layer in densenet201_model.layers:
#         layer.trainable = False
#     #model_2 = Model(inputs=resnet50_model.input, outputs=predictions)
#     combined_model_input = Input(shape = (512,512,3), name = "combined_model_input")
#     m1_predict = model_1(combined_model_input)
#     m2_predict = model_2(combined_model_input)
#     combined = Concatenate()([m1_predict, m2_predict])
# #     fc = Dense(4096, activation='relu',name = "fc1")(combined)
# #     fc = Dense(4096, activation='relu',name = "fc2")(combined)
#     output_layer = Dense(len(CLASSES), activation='softmax',name = "fc3")(combined)
#     model = Model(combined_model_input, output_layer)
# #     model = Model(inputs=([model_1.input, model_2.input]), outputs=merged)
# #     opt = Adam(learning_rate=0.0001)
#     model.compile(optimizer='adam',
#                       loss='sparse_categorical_crossentropy',
#                       metrics=['sparse_categorical_accuracy'])
#     model.summary()
#     tf.keras.utils.plot_model(
#     model)
#     return model

In [ ]:
# tf.keras.utils.plot_model(
#     model)

In [ ]:
cp_callback = ModelCheckpoint(filepath='flower_model1.hdf5',
                              monitor='val_sparse_categorical_accuracy',
                              save_freq='epoch', verbose=1, period=1,
                              save_best_only=True, save_weights_only=True)

early_stopping = EarlyStopping(monitor='val_sparse_categorical_accuracy',
                               verbose=1, patience=5)

In [ ]:
from datetime import datetime, timedelta

start_time = datetime.now()
print('Time now is', start_time)
end_training_by_tdelta = timedelta(seconds=8400)
this_run_file_prefix = start_time.strftime('%Y%m%d_%H%M_')


# EPOCHS = 25
# STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE



# Learning Rate Schedule for Fine Tuning #
# Learning rate schedule for TPU, GPU and CPU.
# Using an LR ramp up because fine-tuning a pre-trained model.
# Starting with a high LR would break the pre-trained weights.

LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = LR_START + (epoch * (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS)
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
rng = [i for i in range(num_epochs)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))



In [ ]:
with strategy.scope():
    batch_size = batch_size * strategy.num_replicas_in_sync
    steps_per_epoch = (12753 / 256)*2
#     model = nerual_network_model()
#     model = desenet121_model()
    model = create_densenet201_model()
#     model = create_resnet50_model()
    history = model.fit(
                train_dataset, 
                validation_data=val_dataset,
                epochs=num_epochs,
                steps_per_epoch=steps_per_epoch,
#                 validation_steps= (3712 / 256)*2,
                callbacks=[cp_callback, early_stopping, lr_callback])

In [ ]:
plt.plot(history.history['loss'], label='train_loss') 
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'], label='train_accuracy')
plt.plot(history.history['val_sparse_categorical_accuracy'], label='val_sparse_categorical_accuracy')
plt.legend() 
plt.show()

In [ ]:
model = create_densenet201_model()
model.load_weights('flower_model1.hdf5')

In [ ]:
results = {'id': [], 'label': []}

In [ ]:
def predict(element):
    image = element[0]
    id_ = tf.keras.backend.get_value(element[1]).decode("utf-8")
    result = list(model.predict(np.array([image]))[0])
    max_pred = max(result)
    result = result.index(max_pred)
    results['id'].append(id_)
    results['label'].append(result) 

In [ ]:
count = 0
for row in test_dataset: 
    image = row[0]
    id_ = tf.keras.backend.get_value(row[1]).decode("utf-8")
    result = list(model.predict(np.array([image]))[0])
    max_pred = max(result) 
    result = result.index(max_pred) 
    results['id'].append(id_)
    results['label'].append(result)
    count += 1 
    if (count % 500) == 0:
        print(f"Finished predicting {count} images") 

In [ ]:
results_df = pd.DataFrame(results)
results_df.to_csv('submission.csv', index=False)


In [ ]:
results_df